In [76]:
import pandas as pd
import numpy as np
from numpy import random
import librosa
import pickle
import joblib
import os
import sys

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import preprocessing


In [77]:
base_path = '/ESC-50'
path = os.path.join(base_path, 'meta/esc50.csv')
csv_input = pd.read_csv(path, encoding="ms932", sep=",")

In [78]:
# data augmentation: add white noise
def add_white_noise(x, rate=0.002):
    return x + rate*np.random.randn(len(x))


# data augmentation: shift sound in timeframe
def shift_sound(x, rate=2):
    return np.roll(x, int(len(x)//rate))


# data augmentation: stretch sound
def stretch_sound(x, rate=1.1):
    input_length = len(x)
    x = librosa.effects.time_stretch(x, rate)
    if len(x) > input_length:
        return x[:input_length]
    else:
        return np.pad(x, (0, max(0, input_length - len(x))), "constant")

In [89]:
path = np.array([])
labels = np.array([])

for i in range(2000):
    if csv_input.category[i] == 'snoring':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'snoring')
    elif csv_input.category[i] == 'washing_machine':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'washing_machine')
    elif csv_input.category[i] == 'vacuum_cleaner':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'vacuum_cleaner')
    elif csv_input.category[i] == 'helicopter':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'helicopter')
    elif csv_input.category[i] == 'chainsaw':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'chainsaw')
    elif csv_input.category[i] == 'engine':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'engine')
    elif csv_input.category[i] == 'airplane':
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'airplane')
    else:
        path = np.append(path, os.path.join(base_path, 'audio/' + csv_input.filename[i]))
        for _ in range(4):
            labels = np.append(labels, 'other')

In [80]:
dataset = []
melspecs = []
mfccs = []
two_sec = 88200 # 2sec

for i in range(2000):

    y_1, sr = librosa.load(path[i], sr = 44100)
    y_1, _ = librosa.effects.trim(y_1, top_db=20)

    if len(y_1) <= two_sec:
        # padding
        pad =  two_sec - len(y_1)
        y_1 = np.concatenate((y_1, np.zeros(pad, dtype=np.float16)))
    else:
        # trimming
        start = random.randint(0, len(y_1) -  two_sec - 1)
        y_1 = y_1[start:start +  two_sec]

    y_2 = add_white_noise(y_1)
    y_3 = shift_sound(y_1)
    y_4 = stretch_sound(y_1)

    dataset.append(y_1)
    dataset.append(y_2)
    dataset.append(y_3)
    dataset.append(y_4)

    # メルスペクトログラムの計算
    melspec_1 = librosa.feature.melspectrogram(y_1, sr)
    melspec_2 = librosa.feature.melspectrogram(y_2, sr)
    melspec_3 = librosa.feature.melspectrogram(y_3, sr)
    melspec_4 = librosa.feature.melspectrogram(y_4, sr)
    melspec_1 = librosa.amplitude_to_db(melspec_1).flatten()
    melspec_2 = librosa.amplitude_to_db(melspec_2).flatten()
    melspec_3 = librosa.amplitude_to_db(melspec_3).flatten()
    melspec_4 = librosa.amplitude_to_db(melspec_4).flatten()

    melspecs.append(melspec_1.astype(np.float16))
    melspecs.append(melspec_2.astype(np.float16))
    melspecs.append(melspec_3.astype(np.float16))
    melspecs.append(melspec_4.astype(np.float16))

    mfcc_1 = np.mean(librosa.feature.mfcc(y=y_1, sr=sr, n_mfcc=40), axis=1)
    mfcc_2 = np.mean(librosa.feature.mfcc(y=y_2, sr=sr, n_mfcc=40), axis=1)
    mfcc_3 = np.mean(librosa.feature.mfcc(y=y_3, sr=sr, n_mfcc=40), axis=1)
    mfcc_4 = np.mean(librosa.feature.mfcc(y=y_4, sr=sr, n_mfcc=40), axis=1)

    mfccs.append(mfcc_1)
    mfccs.append(mfcc_2)
    mfccs.append(mfcc_3)
    mfccs.append(mfcc_4)

    if i % 100 == 0:
        print(i, end=",")
print(2000)

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000


In [81]:
dataset = np.array(dataset)
melspecs = np.array(melspecs)
mfccs = np.array(mfccs)

In [84]:
# 各データの振幅の平均値
mean = np.sqrt(np.mean(dataset**2, axis=1))
print(len(mean))

# 各データのゼロクロス数
zc = np.sum(librosa.zero_crossings(dataset), axis=1)
print(len(zc))

# train_feature学習データ，test_feature予測（テスト）データ
train_feature = np.array([])
train_feature = [np.append(np.append(np.append(np.append(train_feature, mean[i]), zc[i]), melspecs[i]), mfccs[i]) for i in range(8000)]
print(len(train_feature))

8000
8000
8000


In [85]:
x_train, x_test, y_train, y_test = train_test_split(train_feature, labels, test_size=0.2, random_state=0, stratify=labels)
model_1 = LinearSVC()
model_1.fit(x_train, y_train)

c:\Users\pr0gr\RD_club\projects\.venv\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [86]:
print(f"model_1(train): {model_1.score(x_train, y_train)}")
print(f"model_1(test): {model_1.score(x_test, y_test)}")

model_1(train): 0.99703125
model_1(test): 0.905


In [92]:
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(train_feature, labels, test_size=0.1, random_state=0, stratify=labels)
model_2 = LinearSVC()
model_2.fit(x_train_2, y_train_2)

c:\Users\pr0gr\RD_club\projects\.venv\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [93]:
print(f"model_2(train): {model_2.score(x_train_2, y_train_2)}")
print(f"model_2(test): {model_2.score(x_test_2, y_test_2)}")

model_2(train): 0.99875
model_2(test): 0.94375


In [ ]:
import scipy.stats
from sklearn.datasets import load_breast_cancer
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


#条件設定
max_score = 0
SearchMethod = 0
LSVC_grid = {LinearSVC(): {"C": [10 ** i for i in range(-5, 6)],
                           "multi_class": ["ovr", "crammer_singer"],
                           "class_weight": ["balanced"],
                           "random_state": [i for i in range(0, 101)]}}
LSVC_random = {LinearSVC(): {"C": scipy.stats.uniform(0.00001, 1000),
                             "multi_class": ["ovr", "crammer_singer"],
                             "class_weight": ["balanced"],
                             "random_state": scipy.stats.randint(0, 100)}}

#トレーニングデータ、テストデータの分離
train_X, test_X, train_y, test_y = train_test_split(train_feature, labels, random_state=0)

#グリッドサーチ
for model, param in LSVC_grid.items():
    clf = GridSearchCV(model, param)
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    score = f1_score(test_y, pred_y, average="micro")

    if max_score < score:
        max_score = score
        best_param = clf.best_params_
        best_model = model.__class__.__name__

#ランダムサーチ
for model, param in LSVC_random.items():
    clf =RandomizedSearchCV(model, param)
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    score = f1_score(test_y, pred_y, average="micro")

    if max_score < score:
        SearchMethod = 1
        max_score = score
        best_param = clf.best_params_
        best_model = model.__class__.__name__

if SearchMethod == 0:
    print("サーチ方法:グリッドサーチ")
else:
    print("サーチ方法:ランダムサーチ")
print("ベストスコア:{}".format(max_score))
print("モデル:{}".format(best_model))
print("パラメーター:{}".format(best_param))

#ハイパーパラメータを調整しない場合との比較
model = LinearSVC()
model.fit(train_X, train_y)
score = model.score(test_X, test_y)
print("")
print("デフォルトスコア:", score)

In [94]:
C_base_path = '/G2021-maikataA_py'
save_path = os.path.join(C_base_path, 'noise_detection_AI/pickle')
with open(save_path + '/all_model_0.joblib', mode='wb') as f:
    joblib.dump(model_2, f, protocol = 4)

save_path = os.path.join(C_base_path, 'save_AI')
with open(save_path + '/model_predict.joblib', mode='wb') as f:
    joblib.dump(model_2, f, protocol = 4)